In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.subplots as sp
from IPython.display import display
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, LabelEncoder, OneHotEncoder
from datetime import datetime
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)
import folium
from folium.plugins import HeatMap, MousePosition
import random
# from branca.element import Template, MacroElement
from geopy.distance import geodesic
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, BayesianRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error, mean_squared_log_error, make_scorer

In [15]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', 290)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [16]:
df_ = pd.read_csv('hemnet.csv')

In [17]:
df = df_.copy()

In [18]:
def grab_col_names(dataframe, cat_th=10, car_th=300):
    """
    Grab column names based on their types and cardinality.

    Parameters:
    cat_th (int): Threshold for numerical columns to be considered categorical. Default is 10.
    car_th (int): Threshold for categorical columns to be considered cardinal. Default is 20.

    Returns:
    tuple: Lists of categorical columns, numerical columns, and cardinal columns.
    """
    cat_cols = [col for col in dataframe.columns if str(dataframe[col].dtypes) in ['category', 'object', 'bool']]
    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < cat_th and dataframe[col].dtypes in ['int', 'float']]
    cat_but_car = [col for col in dataframe.columns if dataframe[col].nunique() > car_th and str(dataframe[col].dtypes) in ['category', 'object']]
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]
    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes in ['int', 'float', 'int64', 'float64']]
    num_cols = [col for col in num_cols if col not in cat_cols]
    date_cols = [col for col in dataframe.columns if dataframe[col].dtypes == 'datetime64[ns]']

    return cat_cols, num_cols, cat_but_car, date_cols

In [19]:
cat_cols, num_cols, cat_but_car, date_cols = grab_col_names(df)

In [20]:
df.head()

In [21]:
def dataframe_summary(dataframe: pd.DataFrame, response_var : str, head: int = 5, q1: float = 0.25, q2: float = 0.50, q3: float = 0.75, q4: float = 0.95, verbose: bool = False):
    if not isinstance(dataframe, pd.DataFrame):
        raise ValueError("Input must be a pandas DataFrame")

    def print_section(title):
        if verbose:
            print(f"\n{'-' * 20} {title} {'-' * 20}\n")

    cat_cols, num_cols, cat_but_car, date_cols = grab_col_names(dataframe)
    memory_usage = dataframe.memory_usage(deep=True)

    summary_df = pd.DataFrame({
            'Data Type': dataframe.dtypes,
            'Unique Values': dataframe.nunique(),
            'Memory Usage (MB)': memory_usage / 1e6,
            'Null Value Ratio (%)': dataframe.isnull().mean() * 100,
            'Zero Value Ratio (%)': (dataframe == 0).mean() * 100,
            'Skewness': dataframe.select_dtypes(include='number').skew(),
            'Kurtosis': dataframe.select_dtypes(include='number').kurt()
        }).round({
            'Memory Usage (MB)': 3,
            'Null Value Ratio (%)': 3,
            'Zero Value Ratio (%)': 3,
            'Skewness': 3,
            'Kurtosis': 3
        }).reset_index().rename(columns = {'index' : 'Variable'})
    summary_df['Column Type'] = summary_df.Variable.map(
            lambda col: 'Categorical' if col in cat_cols else
                        'Numerical' if col in num_cols else
                        'Cardinal Categorical' if col in cat_but_car else
                        'Date' if col in date_cols else 'Other'
        )

    summary_df['Correlation with Response'] = np.nan

    if response_var in dataframe.columns:
        if pd.api.types.is_numeric_dtype(dataframe[response_var]):
            num_df = dataframe.select_dtypes(include='number')
            if response_var in num_df.columns:
                corr_with_response = num_df.corr()[response_var]
                for var in corr_with_response.index:
                    if var in summary_df['Variable'].values:
                        summary_df.loc[summary_df['Variable'] == var, 'Correlation with Response'] = corr_with_response[var]

    if verbose:
        print_section("Shape of the dataframe")
        print(f"Rows: {dataframe.shape[0]}, Columns: {dataframe.shape[1]}")

        print_section("Column Classification")
        print(f"Categorical Columns: {cat_cols}\n")
        print(f"Numerical Columns: {num_cols}\n")
        print(f"Cardinal Categorical Columns: {cat_but_car}\n")
        print(f"Date Columns: {date_cols}\n")

        print_section("Unique Values, Memory Usage, Null and Zero Value Ratios")

        print(summary_df)

        print_section("Head")
        print(dataframe.head(head))

        print_section("Tail")
        print(dataframe.tail(head))

        print_section("Missing Values")
        missing = dataframe.isnull().sum()
        missing_pct = 100 * dataframe.isnull().sum() / len(dataframe)
        missing_table = pd.concat([missing, missing_pct], axis=1, keys=['Total', 'Percent'])

        missing_columns = missing_table[missing_table['Total'] > 0].sort_values('Total', ascending=False)
        num_missing = len(missing_columns)
        num_cols_pie = 5
        num_rows = -(-num_missing // num_cols_pie)

        fig = make_subplots(
            rows=num_rows,
            cols=num_cols_pie,
            specs=[[{'type': 'domain'}] * num_cols_pie for _ in range(num_rows)],
            subplot_titles=missing_columns.index.tolist()
        )

        for i, col in enumerate(missing_columns.index):
            row = i // num_cols_pie + 1
            col_idx = i % num_cols_pie + 1

            missing_count = missing[col]
            non_missing_count = len(dataframe) - missing_count
            total_count = missing_count + non_missing_count
            pie_data = [missing_count, non_missing_count]
            pie_labels = ['Missing', 'Not Missing']
            pie_percentages = [100 * missing_count / total_count, 100 * non_missing_count / total_count]

            fig.add_trace(
                go.Pie(
                    labels=pie_labels,
                    values=pie_data,
                    hole=0.4,
                    marker=dict(colors=['#ff6f6f', '#6f9eaf']),
                    textinfo='label+percent',
                    name=col
                ),
                row=row,
                col=col_idx
            )

        fig.update_layout(
            title_text="Missing Values - Pie Charts",
            showlegend=True,
            legend=dict(
                x=1.05,
                y=1.0,
                traceorder='normal',
                orientation='v'
            ),
            height=num_rows * 300,
            width=num_cols_pie * 300
        )

        fig.show()
        
        print_section('Mean Values of Response Variable by Categorical Variables')
        n_cols = 5
        n_rows = (len(cat_cols) + n_cols - 1) // n_cols  
        fig = make_subplots(rows=n_rows, cols=n_cols, 
                            subplot_titles=[f'Mean SalePrice by {col}' for col in cat_cols])
        for i, col in enumerate(cat_cols):
            df_filtered = df.groupby(col)['SalePrice'].mean().reset_index()
            fig.add_trace(
                go.Bar(x=df_filtered[col], y=df_filtered['SalePrice'], name='SalePrice', showlegend=False),
                row=i // n_cols + 1, col=i % n_cols + 1
            )    
            mean_value = df_filtered['SalePrice'].mean()
            fig.add_trace(
                go.Scatter(x=[df_filtered[col].min(), df_filtered[col].max()], y=[mean_value, mean_value],
                           mode='lines', line=dict(color='red', dash='dash'), name='Mean SalePrice', showlegend=False),
                row=i // n_cols + 1, col=i % n_cols + 1
            )
        fig.update_layout(
            height=300 * n_rows, 
            width=1500, 
            title_text="Mean SalePrice by Categorical Variables",
            title_x=0.5
        )
        fig.show()

        print_section("Descriptive Statistics")
        datetime_columns = dataframe.select_dtypes(include='datetime64[ns]').columns
        desc = dataframe.drop(datetime_columns, axis = 1).describe([0, q1, q2, q3, q4, 1]).T
        desc['range'] = desc['max'] - desc['min']
        desc['coef_var'] = desc['std'] / desc['mean']
        print(desc)

        print_section("Correlation Heatmap and Correlations with Response Variable")
        #plt.figure(figsize=(24, 20))
        corr = dataframe.select_dtypes(include = 'number').corr()
        mask = np.triu(np.ones_like(corr, dtype = 'bool'))
        plt.figure(figsize = (24, 20))
        sns.heatmap(corr, mask = mask, annot = True, cmap = 'coolwarm', fmt = '.2f')
    else:
        return summary_df

In [22]:
df.columns

In [23]:
summary_df = dataframe_summary(df, 'price')

In [24]:
summary_df.sort_values(by = 'Correlation with Response', ascending = False)

In [26]:
summary_df.to_csv('summary_df.csv', index = False)

In [27]:
def value_counts_ratio(df, columns, plot=False, verbose=False):
    """
    Calculate the value counts and their proportions for specified columns in a DataFrame, and optionally plot pie charts and bar plots.
    Parameters:
    - df (pd.DataFrame): The DataFrame to analyze.
    - columns (list of str): List of column names to calculate value counts for.
    - plot (bool): If True, plot pie charts and bar plots for each column. Default is False.
    - verbose (bool): If True, print the value counts and percentages. Default is False.
    Returns:
    - value_counts_dfs (dict of pd.DataFrame): Dictionary of DataFrames containing value counts and percentages for each column.
    """
    if not isinstance(df, pd.DataFrame):
        raise ValueError("Input must be a pandas DataFrame")
    if not all(isinstance(col, str) for col in columns):
        raise ValueError("Columns must be a list of strings")

    value_counts_dfs = {}
    for col in columns:
        if col not in df.columns:
            raise ValueError(f"Column '{col}' not found in DataFrame")

        value_counts_df = df[col].value_counts(normalize=True).to_frame().reset_index()
        value_counts_df.columns = ['value', 'percentage']
        value_counts_dfs[col] = value_counts_df

        if verbose:
            print(f"\nColumn: {col}")
            print(value_counts_df)

        if plot:
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
            fig.suptitle(f'{col.capitalize()} Proportions', fontsize=16)

            def autopct_format(pct):
                return f'{pct:.1f}%' if pct >= 2 else ''

            wedges, texts, autotexts = ax1.pie(
                value_counts_df['percentage'],
                labels=None,
                autopct=autopct_format,
                colors=sns.color_palette("tab10", len(value_counts_df)),
                textprops={'fontsize': 10}
            )
            ax1.set_title('Pie Chart', fontsize=14)
            ax1.legend(wedges, value_counts_df['value'], title="Values", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))

            sns.barplot(x='value', y='percentage', data=value_counts_df, ax=ax2, palette="tab10")
            ax2.set_title('Bar Plot', fontsize=14)
            ax2.set_xlabel('')
            ax2.set_ylabel('Percentage')
            ax2.set_xticklabels([])

            plt.tight_layout()
            plt.show()

    return value_counts_dfs

In [43]:
def value_counts_ratio(df, column, plot=False, verbose=False, save_image=False, image_path='plot.png'):
    """
    Calculate the value counts and their proportions for a specified column in a DataFrame, and optionally plot a bar plot.
    
    Parameters:
    - df (pd.DataFrame): The DataFrame to analyze.
    - column (str): Column name to calculate value counts for.
    - plot (bool): If True, plot a bar plot for the column. Default is False.
    - verbose (bool): If True, print the value counts and percentages. Default is False.
    - save_image (bool): If True, save the plot as an image file. Default is False.
    - image_path (str): Path to save the image file. Default is 'plot.png'.
    
    Returns:
    - value_counts_df (pd.DataFrame): DataFrame containing value counts and percentages for the column.
    """
    if not isinstance(df, pd.DataFrame):
        raise ValueError("Input must be a pandas DataFrame")
    if not isinstance(column, str):
        raise ValueError("Column must be a string")

    if column not in df.columns:
        raise ValueError(f"Column '{column}' not found in DataFrame")

    # Calculate value counts and percentages
    value_counts_df = df[column].value_counts(normalize=True).to_frame().reset_index()
    value_counts_df.columns = ['value', 'percentage']

    if verbose:
        print(f"\nColumn: {column}")
        print(value_counts_df)

    if plot:
        # Plotting bar plot only
        plt.figure(figsize=(10, 6))
        sns.barplot(x='value', y='percentage', data=value_counts_df, palette="tab10")
        plt.title(f'{column.capitalize()} Proportions', fontsize=16)
        plt.xlabel('')
        plt.ylabel('Percentage')
        plt.xticks(rotation=45)
        plt.tight_layout()
        
        if save_image:
            plt.savefig(image_path, bbox_inches='tight')
            print(f"Plot saved as '{image_path}'")
        else:
            plt.show()

    return value_counts_df

In [31]:
df.head()

In [48]:
value_counts_ratio(df, 'balcony', plot = True, save_image = True, image_path = 'balcony_before.png')

In [53]:
def plot_missing_values_pie_charts(dataframe, save_to_file=False, filename='missing_values_pie_charts.png'):
    missing = dataframe.isnull().sum()
    missing_pct = 100 * dataframe.isnull().sum() / len(dataframe)
    missing_table = pd.concat([missing, missing_pct], axis=1, keys=['Total', 'Percent'])

    missing_columns = missing_table[missing_table['Total'] > 0].sort_values('Total', ascending=False)
    num_missing = len(missing_columns)
    num_cols_pie = 5
    num_rows = -(-num_missing // num_cols_pie)

    fig, axs = plt.subplots(num_rows, num_cols_pie, figsize=(num_cols_pie * 5, num_rows * 5))

    axs = axs.flatten()

    for i, col in enumerate(missing_columns.index):
        missing_count = missing[col]
        non_missing_count = len(dataframe) - missing_count
        total_count = missing_count + non_missing_count
        pie_data = [missing_count, non_missing_count]
        pie_labels = ['Missing', 'Not Missing']

        axs[i].pie(
            pie_data,
            labels=pie_labels,
            colors=['#90EE90', '#D3D3D3'],
            autopct='%1.1f%%',
            startangle=90,
            wedgeprops=dict(width=0.3)
        )
        axs[i].set_title(col, fontsize=12)

    for j in range(num_missing, len(axs)):
        axs[j].axis('off')

    fig.suptitle("Missing Values - Pie Charts", fontsize=16)
    plt.tight_layout(rect=[0, 0, 0.85, 1])

    if save_to_file:
        plt.savefig(filename, bbox_inches='tight')

    plt.show()

In [54]:
plot_missing_values_pie_charts(df, save_to_file = True, filename = 'missing_values_before.png')

In [62]:
def plot_zero_values_pie_charts(dataframe, save_to_file=False, filename='zero_values_pie_charts.png'):
    zero_counts = (dataframe == 0).sum()
    zero_pct = 100 * zero_counts / len(dataframe)
    zero_table = pd.concat([zero_counts, zero_pct], axis=1, keys=['Total', 'Percent'])

    zero_columns = zero_table[zero_table['Total'] > 0].sort_values('Total', ascending=False)
    num_zero = len(zero_columns)
    num_cols_pie = 5
    num_rows = -(-num_zero // num_cols_pie)

    fig, axs = plt.subplots(num_rows, num_cols_pie, figsize=(num_cols_pie * 5, num_rows * 5))

    axs = axs.flatten()

    for i, col in enumerate(zero_columns.index):
        zero_count = zero_counts[col]
        non_zero_count = len(dataframe) - zero_count
        total_count = zero_count + non_zero_count
        pie_data = [zero_count, non_zero_count]
        pie_labels = ['Zero', 'Non-Zero']

        axs[i].pie(
            pie_data,
            labels=pie_labels,
            colors=['#90EE90', '#D3D3D3'],
            autopct='%1.1f%%',
            startangle=90,
            wedgeprops=dict(width=0.3)
        )
        axs[i].set_title(col, fontsize=12)

    for j in range(num_zero, len(axs)):
        axs[j].axis('off')

    fig.suptitle("Zero Values - Pie Charts", fontsize=16)
    plt.tight_layout(rect=[0, 0, 0.85, 1])

    if save_to_file:
        plt.savefig(filename, bbox_inches='tight')

    plt.show()

In [64]:
plot_zero_values_pie_charts(df, save_to_file= True, filename = 'zero_values_before.png')

In [29]:
translation_dict={
'Villa':'House',
'Lägenhet':'Apartment',
'Tomt':'Plot',
'Radhus':'Town house',
'Parhus':'Terraced house',
'Vinterbonatfritidshus':'Winterized holiday home',
'Övrig':'Other',
'Kedjehus':'Linked house',
'Par-/kedje-/radhus':'Row/Terrace/Town house',
'Fritidshus':'Holiday home',
'Gårdutanjordbruk':'Farm without agriculture',
'Gård/skog':'Farm/Forest',
'Gårdmedjordbruk':'Farm with agriculture',
'Fritidsboende':'Holiday accommodation',
'Gårdmedskogsbruk':'Farm with forestry'
}


In [30]:
df.head()

In [34]:
df.columns

In [33]:
value_counts_ratio(df, columns = 'property_type')

In [26]:
df2 = pd.read_csv('hemnet_last.csv')

In [30]:
df2.columns

In [31]:
df2['area']